In [1]:

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data
from torchmetrics.functional.text import bleu_score
import tqdm
import muspy
import src.utils as utils
import src.representation as representation
import src.dataset as dataset
import src.music_x_transformers as music_x_transformers
import pathlib
import src.advUtils as advUtils
import shutil
import re


/home/pcsmet/miniconda3/envs/mtmt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load configurations
train_args = utils.load_json(r"./pre_trained_models/mmt_sod_ape_training_logs.json")
encoding = representation.load_encoding("encoding.json")

sos = encoding["type_code_map"]["start-of-song"]
eos = encoding["type_code_map"]["end-of-song"]
beat_0 = encoding["beat_code_map"][0]
beat_4 = encoding["beat_code_map"][4]
beat_16 = encoding["beat_code_map"][16]

In [3]:
# Load training/testing/demo Data
data_set = advUtils.convert_extract_load(train_args,encoding, midi_dir=pathlib.Path("../preparedData/mid"), json_dir = "./data/test/json",repr_dir="./data/test/repr")

generator1 = torch.Generator().manual_seed(42)
train_set, val_set, test_set = torch.utils.data.random_split(data_set, [400, 40, 10], generator=generator1)
assert (set(re.sub(r"[\[\]\n']", "", pathlib.Path("./val_set.txt").read_text()).split(", "))) == (set([v['name'] for v in val_set])), "mismatch in validation set"
data_loader = torch.utils.data.DataLoader(
    data_set,
    shuffle=True,
    num_workers=1,
    collate_fn=dataset.MusicDataset.collate,
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    shuffle=True,
    num_workers=1,
    collate_fn=dataset.MusicDataset.collate,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    shuffle=True,
    num_workers=1,
    collate_fn=dataset.MusicDataset.collate,
)
valid_loader = torch.utils.data.DataLoader(
    val_set,
    shuffle=True,
    num_workers=1,
    collate_fn=dataset.MusicDataset.collate,
)

In [4]:
set([v['name'] for v in train_set])

{'AlbinoniT-O4N1-1',
 'AlbinoniT-O4N1-2',
 'AlbinoniT-O4N1-3',
 'AlbinoniT-O4N1-4',
 'AlbinoniT-O4N2-1',
 'AlbinoniT-O4N2-2',
 'AlbinoniT-O4N2-4',
 'AlbinoniT-O4N3-1',
 'AlbinoniT-O4N3-2',
 'AlbinoniT-O4N3-3',
 'AlbinoniT-O4N3-4',
 'AlbinoniT-O4N4-2',
 'AlbinoniT-O4N4-4',
 'AlbinoniT-O4N5-1',
 'AlbinoniT-O4N5-2',
 'AlbinoniT-O4N5-3',
 'AlbinoniT-O4N5-4',
 'AlbinoniT-O4N6-1',
 'AlbinoniT-O4N6-2',
 'AlbinoniT-O4N6-4',
 'AlbinoniT-O6N01-1',
 'AlbinoniT-O6N01-2',
 'AlbinoniT-O6N01-3',
 'AlbinoniT-O6N01-4',
 'AlbinoniT-O6N02-1',
 'AlbinoniT-O6N02-2',
 'AlbinoniT-O6N02-4',
 'AlbinoniT-O6N03-1',
 'AlbinoniT-O6N03-2',
 'AlbinoniT-O6N03-3',
 'AlbinoniT-O6N03-4',
 'AlbinoniT-O6N04-1',
 'AlbinoniT-O6N04-2',
 'AlbinoniT-O6N04-3',
 'AlbinoniT-O6N04-4',
 'AlbinoniT-O6N05-1',
 'AlbinoniT-O6N05-3',
 'AlbinoniT-O6N05-4',
 'AlbinoniT-O6N06-1',
 'AlbinoniT-O6N06-2',
 'AlbinoniT-O6N06-3',
 'AlbinoniT-O6N06-4',
 'AlbinoniT-O6N07-1',
 'AlbinoniT-O6N07-2',
 'AlbinoniT-O6N07-3',
 'AlbinoniT-O6N07-4',
 'Albino

In [8]:
# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Creating the model...")
model = music_x_transformers.MusicXTransformer(
    dim=train_args["dim"],
    encoding=encoding,
    depth=train_args["layers"],
    heads=train_args["heads"],
    max_seq_len=train_args["max_seq_len"],
    max_beat=train_args["max_beat"],
    rotary_pos_emb=train_args["rel_pos_emb"],
    use_abs_pos_emb=train_args["abs_pos_emb"],
    emb_dropout=train_args["dropout"],
    attn_dropout=train_args["dropout"],
    ff_dropout=train_args["dropout"],
).to(device)

origModel = music_x_transformers.MusicXTransformer(
    dim=train_args["dim"],
    encoding=encoding,
    depth=train_args["layers"],
    heads=train_args["heads"],
    max_seq_len=train_args["max_seq_len"],
    max_beat=train_args["max_beat"],
    rotary_pos_emb=train_args["rel_pos_emb"],
    use_abs_pos_emb=train_args["abs_pos_emb"],
    emb_dropout=train_args["dropout"],
    attn_dropout=train_args["dropout"],
    ff_dropout=train_args["dropout"],
).to(device)

firstTrain = model
origModel.load_state_dict(torch.load("./pre_trained_models/mmt_sod_ape_best_model.pt", map_location=device))
model.load_state_dict(torch.load("./pre_trained_models/mmt_sod_ape_best_model.pt", map_location=device))

print(f"Loaded the pretrained model weights")
model.eval()

Creating the model...
Loaded the pretrained model weights


MusicXTransformer(
  (decoder): MusicAutoregressiveWrapper(
    (net): MusicTransformerWrapper(
      (token_emb): ModuleList(
        (0): TokenEmbedding(
          (emb): Embedding(5, 512)
        )
        (1): TokenEmbedding(
          (emb): Embedding(257, 512)
        )
        (2): TokenEmbedding(
          (emb): Embedding(13, 512)
        )
        (3): TokenEmbedding(
          (emb): Embedding(129, 512)
        )
        (4): TokenEmbedding(
          (emb): Embedding(33, 512)
        )
        (5): TokenEmbedding(
          (emb): Embedding(65, 512)
        )
      )
      (pos_emb): AbsolutePositionalEmbedding(
        (emb): Embedding(1024, 512)
      )
      (emb_dropout): Dropout(p=0.2, inplace=False)
      (project_emb): Identity()
      (attn_layers): Decoder(
        (layers): ModuleList(
          (0): ModuleList(
            (0): ModuleList(
              (0): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (1): None
              (2): None
    

In [9]:
def generate(n,sample_dir,model=model,modes=["unconditioned"],seq_len=1024,temperature=1,filter_logits="top_k",filter_thresh=0.9):
    with torch.no_grad():
        data_iter = iter(test_loader)
        for i in tqdm.tqdm(range(n), ncols=80):
            batch = next(data_iter)
            print("Generating based on",batch['name'])
            for mode in modes:
                if(mode=="unconditioned"):
                    tgt_start = torch.zeros((1, 1, 6), dtype=torch.long, device=device)
                    tgt_start[:, 0, 0] = sos
                elif(mode=="instrument_informed"):
                    prefix_len = int(np.argmax(batch["seq"][0, :, 1] >= beat_0))
                    tgt_start = batch["seq"][:1, :prefix_len].to(device)
                elif(mode=="4_beat"):
                    cond_len = int(np.argmax(batch["seq"][0, :, 1] >= beat_4))
                    tgt_start = batch["seq"][:1, :cond_len].to(device)
                elif(mode=="16_beat"):
                    cond_len = int(np.argmax(batch["seq"][0, :, 1] >= beat_16))
                    tgt_start = batch["seq"][:1, :cond_len].to(device)
                # Generate new samples
                generated = model.generate(
                    tgt_start,
                    seq_len,
                    eos_token=eos,
                    temperature=temperature,
                    filter_logits_fn=filter_logits,
                    filter_thres=filter_thresh,
                    monotonicity_dim=("type", "beat"),
                )
                generated_np = torch.cat((tgt_start, generated), 1).cpu().numpy()

                # Save the results
                advUtils.save_result(
                    f"{i}_{mode}", generated_np[0], sample_dir, encoding,savecsv=False,savetxt=False,savenpy=False,savepng=False,savejson=False
                )

In [10]:
#Test generation
#generate(2,"./samples",seq_len=50,modes=["unconditioned","instrument_informed"])

<h1>Transfer Learning</h1>

In [11]:
model.decoder.net.to_logits

ModuleList(
  (0): Linear(in_features=512, out_features=5, bias=True)
  (1): Linear(in_features=512, out_features=257, bias=True)
  (2): Linear(in_features=512, out_features=13, bias=True)
  (3): Linear(in_features=512, out_features=129, bias=True)
  (4): Linear(in_features=512, out_features=33, bias=True)
  (5): Linear(in_features=512, out_features=65, bias=True)
)

In [12]:
#Freeze All network layers
for m in model.parameters():
    m.requires_grad = False

new_heads = model.decoder.net.to_logits
new_heads.requires_grad = True

for child in new_heads.children():
    child.reset_parameters()
    for param in child.parameters():
        param.requires_grad = True

print(model.decoder.net.to_logits)


n_parameters = sum(p.numel() for p in model.parameters())
n_trainables = sum(
   p.numel() for p in model.parameters() if p.requires_grad
)
print(f"Number of parameters: {n_parameters}")
print(f"Number of trainable parameters: {n_trainables}")




ModuleList(
  (0): Linear(in_features=512, out_features=5, bias=True)
  (1): Linear(in_features=512, out_features=257, bias=True)
  (2): Linear(in_features=512, out_features=13, bias=True)
  (3): Linear(in_features=512, out_features=129, bias=True)
  (4): Linear(in_features=512, out_features=33, bias=True)
  (5): Linear(in_features=512, out_features=65, bias=True)
)
Number of parameters: 19944950
Number of trainable parameters: 257526


In [13]:
#comparative generation:
generate(2,"./samples",seq_len=50,model=origModel,modes=["unconditioned","instrument_informed","4_beat","16_beat"])
generate(2,"./samples2",seq_len=50,model=firstTrain,modes=["unconditioned","instrument_informed","4_beat","16_beat"])

  0%|                                                     | 0/2 [00:00<?, ?it/s]

Generating based on ['LullyJB-LWV60-A1S01A']


 50%|██████████████████████▌                      | 1/2 [00:03<00:03,  3.07s/it]

Generating based on ['BeckerD-SS-7']


  0%|                                                     | 0/2 [00:00<?, ?it/s]

Generating based on ['BuxtehudeD-BuxWV256-6']


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  3.45it/s]

Generating based on ['BachJS-BWV1080-C16']


In [14]:
#Training the model
def get_lr_multiplier(
    step, warmup_steps, decay_end_steps, decay_end_multiplier
):
    """Return the learning rate multiplier with a warmup and decay schedule.

    The learning rate multiplier starts from 0 and linearly increases to 1
    after `warmup_steps`. After that, it linearly decreases to
    `decay_end_multiplier` until `decay_end_steps` is reached.

    """
    if step < warmup_steps:
        return (step + 1) / warmup_steps
    if step > decay_end_steps:
        return decay_end_multiplier
    position = (step - warmup_steps) / (decay_end_steps - warmup_steps)
    return 1 - (1 - decay_end_multiplier) * position


def train(out_dir,model):
    """Main function."""
    # Parse the command-line arguments

    # Make sure the output directory exists
    pathlib.Path(out_dir).mkdir(exist_ok=True)
    pathlib.Path(out_dir+"/checkpoints").mkdir(exist_ok=True)
    # Get the specified device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

    # Summarize the model
    n_parameters = sum(p.numel() for p in model.parameters())
    n_trainables = sum(
        p.numel() for p in model.parameters() if p.requires_grad
    )
    print(f"Number of parameters: {n_parameters}")
    print(f"Number of trainable parameters: {n_trainables}")

    # Create the optimizer
    optimizer = torch.optim.Adam(model.parameters(), train_args["learning_rate"])
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lr_lambda=lambda step: get_lr_multiplier(
            step,
            train_args["lr_warmup_steps"],
            train_args["lr_decay_steps"],
            train_args["lr_decay_multiplier"],
        ),
    )

    # Create a file to record losses
    loss_csv = open(out_dir+"/loss.csv", "w")
    loss_csv.write(
        "step,train_loss,valid_loss,type_loss,beat_loss,position_loss,"
        "pitch_loss,duration_loss,instrument_loss\n"
    )
    # Initialize variables
    step = 0
    min_val_loss = float("inf")
    if train_args["early_stopping"]:
        count_early_stopping = 0

    # Iterate for the specified number of steps
    train_iterator = iter(train_loader)
    while step < train_args["steps"]:

        # Training
        print(f"Training...")
        model.train()
        recent_losses = []

        for batch in (pbar := tqdm.tqdm(range(train_args["valid_steps"]), ncols=80)):
            # Get next batch
            try:
                batch = next(train_iterator)
            except StopIteration:
                # Reinitialize dataset iterator
                train_iterator = iter(train_loader)
                batch = next(train_iterator)

            # Get input and output pair
            seq = batch["seq"].to(device)
            mask = batch["mask"].to(device)

            # Update the model parameters
            optimizer.zero_grad()
            loss = model(seq, mask=mask)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                model.parameters(), train_args["grad_norm_clip"]
            )
            optimizer.step()
            scheduler.step()

            # Compute the moving average of the loss
            recent_losses.append(float(loss))
            if len(recent_losses) > 10:
                del recent_losses[0]
            train_loss = np.mean(recent_losses)
            pbar.set_postfix(loss=f"{train_loss:8.4f}")

            step += 1

        # Release GPU memory right away
        del seq, mask

        # Validation
        print(f"Validating...")
        model.eval()
        with torch.no_grad():
            total_loss = 0
            total_losses = [0] * 6
            count = 0
            for batch in valid_loader:
                # Get input and output pair
                seq = batch["seq"].to(device)
                mask = batch["mask"].to(device)

                # Pass through the model
                loss, losses = model(seq, return_list=True, mask=mask)

                # Accumulate validation loss
                count += len(batch)
                total_loss += len(batch) * float(loss)
                for idx in range(6):
                    total_losses[idx] += float(losses[idx])
        val_loss = total_loss / count
        individual_losses = [l / count for l in total_losses]
        print(f"Validation loss: {val_loss:.4f}")
        print(
            f"Individual losses: type={individual_losses[0]:.4f}, "
            f"beat: {individual_losses[1]:.4f}, "
            f"position: {individual_losses[2]:.4f}, "
            f"pitch: {individual_losses[3]:.4f}, "
            f"duration: {individual_losses[4]:.4f}, "
            f"instrument: {individual_losses[5]:.4f}"
        )

        # Release GPU memory right away
        del seq, mask

        # Write losses to file
        loss_csv.write(
            f"{step},{train_loss},{val_loss},{individual_losses[0]},"
            f"{individual_losses[1]},{individual_losses[2]},"
            f"{individual_losses[3]},{individual_losses[4]},"
            f"{individual_losses[5]}\n"
        )

        # Save the model
        checkpoint_filename = out_dir+"/checkpoints/"+f"model_{step}.pt"
        torch.save(model.state_dict(), checkpoint_filename)
        print(f"Saved the model to: {checkpoint_filename}")

        # Copy the model if it is the best model so far
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            shutil.copyfile(
                checkpoint_filename,
                out_dir+"/checkpoints/"+"best_model.pt",
            )
            # Reset the early stopping counter if we found a better model
            if train_args["early_stopping"]:
                count_early_stopping = 0
        elif train_args["early_stopping"]:
            # Increment the early stopping counter if no improvement is found
            count_early_stopping += 1

        # Early stopping
        if (
            train_args["early_stopping"]
            and count_early_stopping > train_args["early_stopping_tolerance"]
        ):
            print(
                "Stopped the training for no improvements in "
                f"{train_args['early_stopping_tolerance']} rounds."
            )
            break

    # Log minimum validation loss
    print(f"Minimum validation loss achieved: {min_val_loss}")

    # Save the optimizer states
    optimizer_filename = out_dir+"/checkpoints/"+f"optimizer_{step}.pt"
    torch.save(optimizer.state_dict(), optimizer_filename)
    print(f"Saved the optimizer state to: {optimizer_filename}")

    # Save the scheduler states
    scheduler_filename = out_dir+"/checkpoints/"+f"scheduler_{step}.pt"
    torch.save(scheduler.state_dict(), scheduler_filename)
    print(f"Saved the scheduler state to: {scheduler_filename}")

    # Close the file
    loss_csv.close()

In [15]:
train("experiment",model)

Number of parameters: 19944950
Number of trainable parameters: 257526
Training...


100%|███████████████████████████| 100/100 [00:03<00:00, 31.22it/s, loss=13.5251]

Validating...


Validation loss: 13.0341
Individual losses: type=0.0796, beat: 1.1110, position: 0.2350, pitch: 0.8645, duration: 0.4636, instrument: 0.5048
Saved the model to: experiment/checkpoints/model_100.pt
Training...


100%|███████████████████████████| 100/100 [00:03<00:00, 33.11it/s, loss=11.6848]

...

Validating...


Validation loss: 2.1147
Individual losses: type=0.0020, beat: 0.0266, position: 0.0168, pitch: 0.2972, duration: 0.1613, instrument: 0.0248
Saved the model to: experiment/checkpoints/model_5900.pt
Training...


100%|████████████████████████████| 100/100 [00:03<00:00, 31.10it/s, loss=2.5486]

Validating...


Validation loss: 2.1078
Individual losses: type=0.0020, beat: 0.0259, position: 0.0168, pitch: 0.2967, duration: 0.1610, instrument: 0.0246
Saved the model to: experiment/checkpoints/model_6000.pt
Minimum validation loss achieved: 2.107817289233208
Saved the optimizer state to: experiment/checkpoints/optimizer_6000.pt
Saved the scheduler state to: experiment/checkpoints/scheduler_6000.pt


In [22]:
import re
re.sub(r"[\[\]\\n']", "", pathlib.Path("./val_set.txt").read_text()).split(",\n ")

['BachJS-BWV1080-C07',
 'LullyJB-LWV60-A1S04D',
 'BachJS-BWV787',
 'BuxtehudeD-BuxWV259-5',
 'LullyJB-LWV60-A3S02A',
 'BeckerD-SS-6',
 'BachJS-BWV872-1',
 'LullyJB-LWV60-A2S10-02',
 'AlbioiT-O6N05-2',
 'BeckerD-SS-41',
 'BachJS-BWV875-1',
 'BachJS-BWV858-2',
 'AlbioiT-O6N11-3',
 'BachJS-BWV851-1',
 'BuxtehudeD-BuxWV263-1',
 'LullyJB-LWV60-A2S04A',
 'LullyJB-LWV60-A2S09-01',
 'BuxtehudeD-BuxWV262-2',
 'BachJS-BWV852-2',
 'BachJS-BWV849-1',
 'BeckerD-SS-43',
 'BuxtehudeD-BuxWV259-1',
 'AlbioiT-O6N08-1',
 'BachJS-BWV848-2',
 'BachJS-BWV988-V15',
 'BachJS-BWV861-1',
 'AlbioiT-O4N4-1',
 'AlbioiT-O6N02-3',
 'AlbioiT-O4N4-3',
 'BeckerD-SS-32',
 'BuxtehudeD-BuxWV264-1',
 'AlbioiT-O4N6-3',
 'BeckerD-SS-21',
 'BachJS-BWV885-2',
 'BachJS-BWV870-2',
 'AlbioiT-O6N09-4',
 'LullyJB-LWV60-A0-09',
 'BuxtehudeD-BuxWV265-1',
 'BachJS-BWV988-V11',
 'BuxtehudeD-BuxWV262-4']

In [25]:
print(len(val_set))

40


In [27]:
set([v['name'] for v in val_set])

{'AlbinoniT-O4N4-1',
 'AlbinoniT-O4N4-3',
 'AlbinoniT-O4N6-3',
 'AlbinoniT-O6N02-3',
 'AlbinoniT-O6N05-2',
 'AlbinoniT-O6N08-1',
 'AlbinoniT-O6N09-4',
 'AlbinoniT-O6N11-3',
 'BachJS-BWV1080-C07',
 'BachJS-BWV787',
 'BachJS-BWV848-2',
 'BachJS-BWV849-1',
 'BachJS-BWV851-1',
 'BachJS-BWV852-2',
 'BachJS-BWV858-2',
 'BachJS-BWV861-1',
 'BachJS-BWV870-2',
 'BachJS-BWV872-1',
 'BachJS-BWV875-1',
 'BachJS-BWV885-2',
 'BachJS-BWV988-V11',
 'BachJS-BWV988-V15',
 'BeckerD-SS-21',
 'BeckerD-SS-32',
 'BeckerD-SS-41',
 'BeckerD-SS-43',
 'BeckerD-SS-6',
 'BuxtehudeD-BuxWV259-1',
 'BuxtehudeD-BuxWV259-5',
 'BuxtehudeD-BuxWV262-2',
 'BuxtehudeD-BuxWV262-4',
 'BuxtehudeD-BuxWV263-1',
 'BuxtehudeD-BuxWV264-1',
 'BuxtehudeD-BuxWV265-1',
 'LullyJB-LWV60-A0-09',
 'LullyJB-LWV60-A1S04D',
 'LullyJB-LWV60-A2S04A',
 'LullyJB-LWV60-A2S09-01',
 'LullyJB-LWV60-A2S10-02',
 'LullyJB-LWV60-A3S02A'}

In [16]:

generate(2,"./samples2",seq_len=50,model=model,modes=["unconditioned","instrument_informed","4_beat","16_beat"])
# origModel

  0%|                                                     | 0/2 [00:00<?, ?it/s]

Generating based on ['BuxtehudeD-BuxWV256-6']


 50%|██████████████████████▌                      | 1/2 [00:02<00:02,  2.40s/it]

Generating based on ['BachJS-BWV776']


100%|█████████████████████████████████████████████| 2/2 [00:04<00:00,  2.25s/it]


In [17]:
xi = seq[:, :-1]
xo = seq[:, 1:]
xi, xo

NameError: name 'seq' is not defined

In [ ]:
# model()
batch = next(iter(train_loader))
seq = batch["seq"].to(device)
mask = batch["mask"].to(device)

# Pass through the model
loss, losses = model(seq, return_list=True, mask=mask)
losses

KeyboardInterrupt: 

In [ ]:
seq

tensor([[[  0,   0,   0,   0,   0,   0],
         [  1,   0,   0,   0,   0,  26],
         [  1,   0,   0,   0,   0,  27],
         ...,
         [  3, 159,  10,  82,   3,  26],
         [  3, 160,   1,  53,  20,  28],
         [  4,   0,   0,   0,   0,   0]]], device='cuda:0')

In [ ]:
str(seq)

"tensor([[[  0,   0,   0,   0,   0,   0],\n         [  1,   0,   0,   0,   0,  26],\n         [  1,   0,   0,   0,   0,  27],\n         ...,\n         [  3, 159,  10,  82,   3,  26],\n         [  3, 160,   1,  53,  20,  28],\n         [  4,   0,   0,   0,   0,   0]]], device='cuda:0')"

In [ ]:
bleu_score([str(seq)], [str(seq)])

tensor(1.)

In [ ]:
list(batch.keys())

['name', 'seq', 'seq_len', 'mask']